In [1]:
# # # !pip install sqlalchemy
# # # !pip install psycopg2-binary
# # # !pip install Flask-SQLAlchemy
# # # !pip3 install sqlalchemy
# # # !pip3 install psycopg2-binary
# # # !pip3 install Flask-SQLAlchemy

# !conda install -y sqlalchemy
# !conda install -y psycopg2-binary

# !conda install -y sqlalchemy
# !conda install -y psycopg2-binary
# !conda install -y Flask-SQLAlchemy

# !pip install python-dotenv

import os

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
host = 'localhost'
port = 5432
database = 'northwind'
user = os.getenv('POSTGRES_USER')
password = os.getenv('POSTGRES_PASSWORD') # set in .env file
password = 'changeme'
uri = f'postgresql://{user}:{password}@{host}:{port}/{database}'

In [4]:
from sqlalchemy import create_engine

engine = create_engine(uri, echo=False)

In [24]:
import pandas as pd

query = 'SELECT c."categoryID", c."categoryName", c."description" FROM categories AS c;'

df = pd.read_sql(query, engine)
df.head()

,categoryID,categoryName,description
0,1,Beverages,Soft drinks coffees teas beers and ales
1,2,Condiments,Sweet and savory sauces relishes spreads and s...
2,3,Confections,Desserts candies and sweet breads
3,4,Dairy Products,Cheeses
4,5,Grains/Cereals,Breads crackers pasta and cereal


In [29]:
ratings = pd.read_csv('../data/all_penguins_clean.csv')
ratings.to_sql('all_penguins_clean', engine, method='multi', chunksize=1000)

ValueError: Table 'all_penguins_clean' already exists.